### QA 
1. In dataset02 what the data number means?
2.

### 1. Load molecules and environmental variable datas

In [26]:
## Load the molecular data (.pkl) for molecules with Targeted BDE & BDFE 

import os
import pickle
import networkx as nx
from torch_geometric.utils import from_networkx

folder_path = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles"

# Get all .pkl file paths in the folder
pkl_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".pkl")]

# Load all graphs
graph_list = []
for file_path in pkl_files:
    with open(file_path, "rb") as file:
        graph = pickle.load(file)  # Load each .pkl graph
        graph_list.append(graph)  # Store in a list

print(f"Loaded {len(graph_list)} graphs")


Loaded 20 graphs


In [28]:
## 检查

# Select the first graph for visualization
nx_graph = graph_list[0]

# Convert node data to DataFrame
node_data = pd.DataFrame.from_dict(dict(nx_graph.nodes(data=True)), orient="index")

# Convert edge data to DataFrame
edge_data = pd.DataFrame.from_records([(u, v, d) for u, v, d in nx_graph.edges(data=True)], 
                                      columns=["source", "target", "attributes"])

# Expand edge attributes
if not edge_data.empty and isinstance(edge_data.iloc[0]["attributes"], dict):
    attr_df = edge_data["attributes"].apply(pd.Series)  # Convert dict to columns
    edge_data = pd.concat([edge_data.drop(columns=["attributes"]), attr_df], axis=1)

# Display DataFrames
from IPython.display import display
display(node_data)
display(edge_data)



,symbol,rdkit_idx
0,C,0.0
1,C,1.0
2,C,2.0
3,C,3.0
4,C,4.0
5,C,5.0
6,C,6.0
7,C,7.0
8,C,8.0
9,C,9.0


,source,target,bond_index,bde_pred,bdfe_pred
0,0,1,0.0,89.382645,75.711845
1,0,H_23,25.0,100.077171,91.049126
2,1,2,1.0,85.872452,71.412849
3,1,H_27,29.0,97.163109,87.689636
4,2,3,2.0,85.041306,70.000282
5,2,H_28,30.0,95.392189,86.257263
6,3,4,3.0,83.115479,66.995277
7,3,H_30,32.0,94.518471,84.748619
8,4,5,NaN,NaN,NaN
9,4,10,NaN,NaN,NaN


备注：os.listdir... 列出文件夹中文件名称，迭代获取，保留.pkl,output list
     "rb"对口pkl二进制

In [29]:
## Convert Graphs to PyTorch Geometric Format for GNN model
# 方便训练和GNN计算，转化为PyG

import os
import pickle
import torch
import networkx as nx
from torch_geometric.data import Data

folder_path = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles"

# Get all .pkl file paths
pkl_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".pkl")]

# List to store PyTorch Geometric graphs
pytorch_graphs = []

# Loop through all .pkl files 
# 分别提取节点和边的数据重构graph

for i, file_path in enumerate(pkl_files):
    with open(file_path, "rb") as file:
        nx_graph = pickle.load(file)  # Load the NetworkX graph
        
        if isinstance(nx_graph, nx.Graph):
            # **Manually relabel nodes to integers**
            mapping = {node: idx for idx, node in enumerate(nx_graph.nodes())}
            nx_graph = nx.relabel_nodes(nx_graph, mapping)  # Apply new integer labels

            # Convert edges to PyG format
            edge_index = torch.tensor(list(nx_graph.edges()), dtype=torch.long).T

            # Extract node features if available
            first_node = next(iter(nx_graph.nodes))  # Get the first node
            if "node_features" in nx_graph.nodes[first_node]:  # Check if node has features
                node_features = torch.tensor(
                    [nx_graph.nodes[n]["node_features"] for n in nx_graph.nodes], dtype=torch.float
                )
            else:
                node_features = None  # No node features available

            # Extract edge features if available
            first_edge = next(iter(nx_graph.edges(data=True)), None)
            if first_edge and "edge_features" in first_edge[2]:  # Check if edges have features
                edge_features = torch.tensor(
                    [nx_graph[u][v]["edge_features"] for u, v in nx_graph.edges()], dtype=torch.float
                )
            else:
                edge_features = None  # No edge features available

            # Create PyG graph
            pyg_graph = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)

            # Store the processed graph
            pytorch_graphs.append(pyg_graph)
            print(f"Converted Graph {i+1}: {len(nx_graph.nodes)} nodes, {len(nx_graph.edges)} edges")

print(f"Total converted graphs: {len(pytorch_graphs)}")


Converted Graph 1: 42 nodes, 44 edges
Converted Graph 2: 43 nodes, 45 edges
Converted Graph 3: 52 nodes, 56 edges
Converted Graph 4: 54 nodes, 58 edges
Converted Graph 5: 56 nodes, 60 edges
Converted Graph 6: 56 nodes, 60 edges
Converted Graph 7: 58 nodes, 62 edges
Converted Graph 8: 58 nodes, 62 edges
Converted Graph 9: 60 nodes, 64 edges
Converted Graph 10: 60 nodes, 64 edges
Converted Graph 11: 62 nodes, 66 edges
Converted Graph 12: 62 nodes, 66 edges
Converted Graph 13: 46 nodes, 48 edges
Converted Graph 14: 48 nodes, 50 edges
Converted Graph 15: 52 nodes, 54 edges
Converted Graph 16: 52 nodes, 54 edges
Converted Graph 17: 53 nodes, 55 edges
Converted Graph 18: 53 nodes, 55 edges
Converted Graph 19: 48 nodes, 52 edges
Converted Graph 20: 48 nodes, 52 edges
Total converted graphs: 20


In [40]:
##  Load Environmental Data from Excel
import pandas as pd

# Load the environmental data
df = pd.read_excel(r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles\dataset02.xlsx", engine='openpyxl')


# Convert to tensor (assuming numerical data)
env_data = torch.tensor(df.values, dtype=torch.float)


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [41]:
print(df.dtypes)


data number         float64
temperature         float64
seawater             object
concentration         int64
time                  int64
component            object
BDE                 float64
BDFE                float64
energy              float64
degradation_rate    float64
dtype: object
